In [1]:
import pandas as pd
import rdflib
from rdflib.namespace import OWL

In [2]:
pairs = pd.read_csv('results/sameAs_pairs.tsv', sep='\t')

Construct owl:sameAs clusters.

In [3]:
uri_to_cluster = dict()
clusters = []

for _, r in pairs.iterrows():
    s = r['s']
    o = r['o']
    cluster_no = uri_to_cluster.get(s, None)
    if cluster_no is None:
        cluster_no = uri_to_cluster.get(o, None)

    if cluster_no is None:
        uri_to_cluster[s] = len(clusters)
        uri_to_cluster[o] = len(clusters)
        clusters.append({s, o})
    else:
        uri_to_cluster[s] = cluster_no
        uri_to_cluster[o] = cluster_no
        clusters[cluster_no].add(s)
        clusters[cluster_no].add(o)

Build an RDF graph, while pruning the external URIs (we won't need them later).

In [4]:
g = rdflib.Graph()
for cluster in clusters:
    to_include = [uri for uri in cluster if uri.startswith('https://cso.kmi.open.ac.uk/topics/')]
    for i in to_include:
        for j in to_include:
            if i != j:
                g.add((
                    rdflib.URIRef(i),
                    OWL.sameAs,
                    rdflib.URIRef(j)
                ))

In [5]:
g.serialize(format='ttl', destination='results/sameAs_inferred.ttl')

<Graph identifier=Nbaee6b55f7604f429383bb40165220fa (<class 'rdflib.graph.Graph'>)>